In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
batch_size=64
test_batch_size=64
lr = 0.001
momentum = 0.001
epochs = 10
log_interval = 200

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(5*5*64, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(x.size(0), -1)
        x = self.drop_out(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    


In [4]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [5]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [6]:
use_cuda = torch.cuda.is_available()

In [7]:
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cpu')

In [8]:
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
kwargs

{}

In [9]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)


0it [00:00, ?it/s]

9920512it [00:03, 3111553.18it/s]                             


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz


0it [00:00, ?it/s]

32768it [00:00, 101513.07it/s]           
0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:01, 1384246.47it/s]                           
0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 33585.46it/s]            


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [10]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

for epoch in range(1,epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302917
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.260669
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.162899
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.954748
Train Epoch: 1 [51200/60000 (85%)]	Loss: 1.789925

Test set: Average loss: 1.2702, Accuracy: 7399/10000 (74%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.399159
Train Epoch: 2 [12800/60000 (21%)]	Loss: 1.201222
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.868683
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.704095
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.525111

Test set: Average loss: 0.5336, Accuracy: 8580/10000 (86%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.805290
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.570154
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.592398
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.658759
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.422037

Test set: Average loss: 0.4062, Accuracy: 8828/10000 (88%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.600188
Train Epoch: 4 [12800/60000 (21%)]	Lo